In [177]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from rdkit import Chem
import re
import os

In [178]:
# df = pd.read_csv('data/raw/bioassays/PROTACXN_P35372_bioassay.csv')
# receptor_type = "mu"
# df = pd.read_csv('data/raw/bioassays/PROTACXN_P41143_bioassay.csv')
# receptor_type = "delta"
# df = pd.read_csv('data/raw/bioassays/PROTACXN_P41145_bioassay.csv')
# receptor_type = "kappa"
# df = pd.read_csv('data/raw/bioassays/PROTACXN_P41146_bioassay.csv')
# receptor_type = "nociception"
# df = pd.read_csv('data/raw/bioassays/PROTACXN_P14416_bioassay.csv')
# receptor_type = "D2LDR"
# df = pd.read_csv('data/raw/bioassays/PROTACXN_P35462_bioassay.csv')
# receptor_type = "D3DR"
# df = pd.read_csv('data/raw/bioassays/PROTACXN_Q8TCU5_bioassay.csv')
# receptor_type = "NMDAR"
df = pd.read_csv('data/raw/bioassays/PROTACXN_P42261_bioassay.csv')
receptor_type = "AMPAR"
df.columns

Index(['aid', 'aidtype', 'aidname', 'aiddesc', 'aidsrcid', 'aidsrcname',
       'aidextid', 'aidmdate', 'cids', 'sids', 'geneids', 'aidcategories',
       'protacxns', 'depcatg', 'pmids', 'rnai', 'ecs', 'repacxns', 'taxids',
       'cellids', 'targettaxid', 'annotation'],
      dtype='object')

In [179]:
outdir = "bioassays/" + receptor_type + "/"
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [180]:
def filter(x):
    x = x.lower()
    if "partial agonist" in x or "partial-agonist" in x:
        return 1
    if "partial agonists" in x or "partial-agonists" in x:
        return 1
    if "partial agonism" in x or "partial-agonism" in x:
        return 1
    else:
        return 0
    
df['partial_agonist_desc']= df.aiddesc.apply(filter)
df['partial_agonist_aidname']= df.aidname.apply(filter)

In [181]:
def filter(x):
    x = x.lower()
    if "agonist" in x:
        return 1
    if "agonists" in x:
        return 1
    if "agonism" in x:
        return 1
    else:
        return 0
    
df['agonist_desc']= df.aiddesc.apply(filter)
df['agonist_aidname']= df.aidname.apply(filter)

In [182]:
def filter(x):
    x = x.lower()
    if "antagonist" in x:
        return 1
    if "antagonists" in x:
        return 1
    if "antagonism" in x:
        return 1
    else:
        return 0
    
df['antagonist_desc']= df.aiddesc.apply(filter)
df['antagonist_aidname']= df.aidname.apply(filter)

In [183]:
compile_index = set([x for x in df.index])

In [184]:
df_test = df.loc[((df['partial_agonist_desc'] == 1) | (df['partial_agonist_aidname'] == 1)) & 
                    ((df['agonist_desc'] == 0) | (df['agonist_aidname'] == 0)) & 
                    ((df['antagonist_desc'] == 0) | (df['antagonist_aidname'] == 0))]

compile_index = compile_index - set(df_test.index)
df_test.set_index('aid', inplace=True)
df_test.to_csv(os.path.join(outdir, receptor_type + '_partial_agonist.csv'))

In [185]:
df_test = df.loc[((df['partial_agonist_desc'] == 0) | (df['partial_agonist_aidname'] == 0)) &
                ((df['agonist_desc'] == 1) | (df['agonist_aidname'] == 1)) &
                ((df['antagonist_desc'] == 0) | (df['antagonist_aidname'] == 0))]

compile_index = compile_index - set(df_test.index)
df_test.set_index('aid', inplace=True)
df_test.to_csv(os.path.join(outdir, receptor_type + '_agonist.csv'))

In [186]:
df_test = df.loc[((df['partial_agonist_desc'] == 0) | (df['partial_agonist_aidname'] == 0)) &
                ((df['agonist_desc'] == 0) | (df['agonist_aidname'] == 0)) &
                ((df['antagonist_desc'] == 1) | (df['antagonist_aidname'] == 1))]

compile_index = compile_index - set(df_test.index)
df_test.set_index('aid', inplace=True)
df_test.to_csv(os.path.join(outdir, receptor_type + '_antagonist.csv'))

In [187]:
compile_index = list(compile_index)
compile_index.sort()

df_test = df.iloc[compile_index]
df_test.to_csv(os.path.join(outdir, receptor_type + '_other.csv'))